# Exploration bivariée et multivariée – Jeu de données Weigh Lifestyle
### Objectifs
- Quantifier les relations conjointes entre les variables comportementales, nutritionnelles et physiologiques.
- Prioriser les variables pour la modélisation en mesurant leur association avec la cible de poids.
- Documenter les distributions catégorielles afin de mettre en évidence les archétypes de style de vie dominants.

### Ressources de données
- Jeu de données nettoyé : `../data/dataset_cleaned.csv`
- Graphiques générés : `../plots/matrice_corr/`, `../plots/correlation/`



## Plan d'analyse
### Étape 1 – Charger les observations préparées
- Lire le jeu de données nettoyé depuis le disque et conserver des chemins partagés pour chaque étape ultérieure.

### Étape 2 – Construire la matrice de corrélation des variables
- Calculer les corrélations par paires pour toutes les variables numériques, y compris la cible de poids, afin de visualiser l'ensemble des relations.

### Étape 3 – Visualiser la structure de corrélation
- Tracer la matrice de corrélation complète et l'enregistrer dans le dossier dédié pour examen.

### Étape 4 – Mesurer les associations variable–cible
- Calculer les corrélations de Spearman et exporter des nuages de points pour inspecter les tendances monotones vis-à-vis de `Weight (kg)`.

### Étape 5 – Profiler les variables catégorielles
- Générer des tableaux de fréquences détaillés et synthétiques pour documenter les catégories de style de vie dominantes dans la cohorte.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.stats import spearmanr

# Définir explicitement les ressources partagées pour garantir des réexécutions reproductibles quel que soit l'environnement.
DATA_DIRECTORY: Path = Path("..") / "data"
CLEANED_DATASET_PATH: Path = DATA_DIRECTORY / "dataset_cleaned.csv"
PLOTS_MATRIX_DIR: Path = Path("..") / "plots" / "matrice_corr"
PLOTS_CORRELATION_DIR: Path = Path("..") / "plots" / "correlation"
PLOTS_MATRIX_PATH: Path = PLOTS_MATRIX_DIR / "correlation_matrix.png"


In [2]:
def compute_correlation_matrix(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute correlation matrix for all numeric features including target variable.
    
    Args:
        df: DataFrame with all features including target
        
    Returns:
        Correlation matrix including Weight (kg) column
    """
    # Ne conserver que les colonnes numériques afin d'éviter les erreurs de type.
    numeric_df = df.select_dtypes(include=[np.number])
    
    return numeric_df.corr()

In [3]:
# Charger le jeu de données nettoyé via un chemin centralisé pour assurer la cohérence entre les notebooks.
df = pd.read_csv(CLEANED_DATASET_PATH)

# Calculer la matrice de corrélation
corr_matrix = compute_correlation_matrix(df)
print(f"Correlation matrix shape: {corr_matrix.shape}")

# Afficher uniquement les couples uniques (triangle inférieur) triés par corrélation absolue décroissante.
mask = np.tril(np.ones_like(corr_matrix, dtype=bool), k=-1)
corr_pairs = corr_matrix.where(mask).stack().reset_index()
corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

corr_pairs_sorted = corr_pairs.sort_values(by='Correlation', key=lambda s: s.abs(), ascending=False)
# Limiter l'aperçu aux 15 corrélations les plus fortes pour conserver l'information clé.
top_corr_pairs = corr_pairs_sorted.head(15)
top_corr_pairs


Correlation matrix shape: (18, 18)


,Feature_1,Feature_2,Correlation
139,pct_maxHR,Avg_BPM,0.840785
27,Experience_Level,Workout_Frequency (days/week),0.836440
26,Experience_Level,Session_Duration (hours),0.758127
20,Workout_Frequency (days/week),Session_Duration (hours),0.638039
138,pct_maxHR,Max_BPM,-0.559991
71,cholesterol_mg,Session_Duration (hours),0.092592
112,cook_time_min,Experience_Level,-0.087249
111,cook_time_min,Workout_Frequency (days/week),-0.082863
54,sugar_g,Physical exercise,0.069642
2,Max_BPM,Weight (kg),0.067038


ATTENTION : décision de supprimer les variables : cal_from_macros / pct_HRR / Carbs / Protein / Fasts car elles sont trop corrélées.

In [4]:
def plot_correlation_matrix(corr_matrix: pd.DataFrame, output_path: Path) -> None:
    """
    Render a complete heatmap of the feature correlation matrix and save to disk.
    
    Args:
        corr_matrix: Correlation matrix (all numeric features)
        output_path: File path where the plot should be saved
    """
    plt.figure(figsize=(14, 12))
    
    # Utiliser une palette divergente pour mettre en évidence les corrélations positives et négatives.
    sns.heatmap(
        corr_matrix,
        annot=False,  # Too many values would clutter the plot
        cmap='coolwarm',
        center=0,
        vmin=-1,
        vmax=1,
        square=True,
        linewidths=0.5,
        cbar_kws={'shrink': 0.8, 'label': 'Correlation coefficient'}
    )
    
    plt.title('Correlation Matrix - All Numeric Features', fontsize=16, pad=20)
    plt.tight_layout()
    
    # S'assurer que le répertoire existe pour éviter les erreurs d'enregistrement sur un nouvel environnement.
    output_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Correlation matrix saved to: {output_path}")

In [5]:
# Tracer et enregistrer la matrice de corrélation en utilisant le chemin partagé défini.
plot_correlation_matrix(corr_matrix, PLOTS_MATRIX_PATH)


Correlation matrix saved to: ../plots/matrice_corr/correlation_matrix.png


In [6]:
def plot_spearman_correlation_with_target(df: pd.DataFrame, target_column: str, output_dir: Path) -> pd.DataFrame:
    """
    Compute Spearman correlation between each numeric feature and target variable,
    then create scatter plots for each relationship.
    
    Args:
        df: DataFrame with all features including target
        target_column: Name of the target variable column
        output_dir: Directory where correlation plots should be persisted
        
    Returns:
        DataFrame with features and their Spearman correlation with target
    """
        
    # Ne conserver que les colonnes numériques pour éviter les erreurs lors de l'appel aux fonctions de scipy.
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Exclure la cible des variables pour que les corrélations mettent en avant les prédicteurs potentiels.
    features = numeric_df.drop(columns=[target_column], errors='ignore')
    
    # Quantifier les relations monotones afin de prioriser les signaux les plus forts.
    correlations = []
    for col in features.columns:
        corr, pvalue = spearmanr(df[col], df[target_column], nan_policy='omit')
        correlations.append({
            'Feature': col,
            'Spearman_Correlation': corr,
            'P_Value': pvalue
        })
    
    corr_df = pd.DataFrame(correlations).sort_values(by='Spearman_Correlation', key=lambda s: s.abs(), ascending=False)
    
    # S'assurer de l'existence du répertoire afin que les traitements batch ne tombent pas en échec sur un espace de travail vierge.
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Visualiser chaque relation pour repérer d'éventuels schémas non linéaires ou une hétéroscédasticité.
    for _, row in corr_df.iterrows():
        feature = str(row['Feature'])
        corr_value = float(row['Spearman_Correlation'])
        
        plt.figure(figsize=(10, 6))
        plt.scatter(df[feature], df[target_column], alpha=0.5, s=20)
        plt.xlabel(feature, fontsize=12)
        plt.ylabel(target_column, fontsize=12)
        plt.title(f'{feature} vs {target_column}\\nSpearman ρ = {corr_value:.3f}', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # Assainir les noms de fichiers pour éviter les problèmes entre systèmes d'exploitation.
        safe_filename = feature.replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '')
        plot_path = output_dir / f'{safe_filename}_vs_Weight.png'
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        plt.close()
    
    print(f"Created {len(corr_df)} correlation plots in {output_dir}")
    
    return corr_df


In [7]:
# Calculer et tracer les corrélations de Spearman avec la variable cible
spearman_results = plot_spearman_correlation_with_target(
    df=df,
    target_column='Weight (kg)',
    output_dir=PLOTS_CORRELATION_DIR
)

# Afficher les corrélations avec des p-valeurs arrondies
spearman_results_display = spearman_results.copy()
spearman_results_display['P_Value'] = spearman_results_display['P_Value'].round(4)
spearman_results_display_sorted = spearman_results_display.sort_values(
    by='Spearman_Correlation',
    key=lambda s: s.abs(),
    ascending=False
)
# Limiter l'affichage aux 15 relations les plus fortes pour synthétiser l'information essentielle.
spearman_results_display_sorted.head(15)


Created 17 correlation plots in ../plots/correlation


,Feature,Spearman_Correlation,P_Value
6,Experience_Level,0.072177,0.0000
1,Max_BPM,0.063006,0.0000
14,cook_time_min,-0.049948,0.0000
5,Workout_Frequency (days/week),0.048058,0.0000
7,Daily meals frequency,0.045625,0.0000
12,serving_size_g,0.044526,0.0000
4,Session_Duration (hours),0.043467,0.0000
0,Age,-0.041913,0.0000
3,Resting_BPM,-0.034753,0.0000
13,prep_time_min,-0.026064,0.0002


Les calories sont très corrélées à la cible, avec une relation quasi linéaire. Elles peuvent transmettre beaucoup d'information aux modèles, mais cela reflète le style de vie sans renseigner la taille de la personne ; nous conservons donc cette variable.

# Analyse des variables catégorielles

## 1. Statistiques descriptives univariées


In [8]:
def compute_categorical_frequencies(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Compute absolute and relative frequencies for a categorical variable.
    
    Args:
        df: DataFrame containing the data
        column: Name of the categorical column to analyze
        
    Returns:
        DataFrame with absolute frequency, relative frequency, and cumulative frequency
    """
    # Capturer les effectifs absolus afin de conserver l'échelle réelle de la population.
    freq_abs = df[column].value_counts().sort_index()
    
    # Convertir les effectifs en pourcentages pour rendre les catégories comparables entre échantillons.
    freq_rel = (freq_abs / len(df) * 100).round(2)
    
    # Suivre la part cumulée pour montrer la vitesse à laquelle les catégories saturent la population.
    freq_cum = freq_rel.cumsum().round(2)
    
    # Rassembler les métriques dans une table unique pour alimenter directement les cellules de reporting.
    result = pd.DataFrame({
        'Fréquence absolue': freq_abs,
        'Fréquence relative (%)': freq_rel,
        'Fréquence cumulative (%)': freq_cum
    })
    
    # Ajouter les totaux pour faciliter les vérifications rapides lors des présentations.
    result.loc['TOTAL'] = [freq_abs.sum(), 100.0, 100.0]
    
    return result


In [9]:
def analyze_categorical_variables(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute descriptive statistics for all categorical variables.
    
    Args:
        df: DataFrame containing the data
        
    Returns:
        DataFrame with mode, unique values count, and most frequent value percentage
    """
    # Limiter l'analyse aux colonnes catégorielles car les numériques disposent d'un traitement spécifique.
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    
    stats = []
    for col in categorical_cols:
        mode_value = df[col].mode()[0]  # Favour the first mode to keep the summary deterministic.
        n_unique = df[col].nunique()
        mode_freq = (df[col] == mode_value).sum()
        mode_pct = (mode_freq / len(df) * 100).round(2)
        
        stats.append({
            'Variable': col,
            'Mode': mode_value,
            'Fréquence du mode': mode_freq,
            'Fréquence du mode (%)': mode_pct,
            'Valeurs uniques': n_unique
        })
    
    return pd.DataFrame(stats)


In [10]:
# Mettre en avant les fréquences détaillées de chaque variable catégorielle pour détecter les déséquilibres.
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

for col in categorical_cols:
    print(f"\n{'='*80}")
    print(f"Variable: {col}")
    print('='*80)
    freq_table = compute_categorical_frequencies(df, col)
    if len(freq_table) > 11:
        freq_preview = pd.concat([freq_table.head(10), freq_table.tail(1)])
    else:
        freq_preview = freq_table
    print(freq_preview)
    if len(freq_table) > len(freq_preview):
        print(f"... {len(freq_table) - len(freq_preview)} catégories supplémentaires non affichées")
    print(f"\nMode: {df[col].mode()[0]}")
    print(f"Nombre de valeurs uniques: {df[col].nunique()}")



Variable: Gender
        Fréquence absolue  Fréquence relative (%)  Fréquence cumulative (%)
Gender                                                                     
Female            10028.0                   50.14                     50.14
Male               9972.0                   49.86                    100.00
TOTAL             20000.0                  100.00                    100.00

Mode: Female
Nombre de valeurs uniques: 2

Variable: Workout_Type
              Fréquence absolue  Fréquence relative (%)  \
Workout_Type                                              
Cardio                   4923.0                   24.62   
HIIT                     4974.0                   24.87   
Strength                 5071.0                   25.36   
Yoga                     5032.0                   25.16   
TOTAL                   20000.0                  100.00   

              Fréquence cumulative (%)  
Workout_Type                            
Cardio                           24.62 

In [11]:
# Construire une vue agrégée pour comparer d'un coup d'œil la domination des catégories.
categorical_stats = analyze_categorical_variables(df)
print("=== Vue d'ensemble des variables catégorielles ===\n")
categorical_stats_sorted = categorical_stats.sort_values(
    by='Fréquence du mode (%)',
    ascending=False
)
categorical_stats_sorted.head(15)


=== Vue d'ensemble des variables catégorielles ===



,Variable,Mode,Fréquence du mode,Fréquence du mode (%),Valeurs uniques
0,Gender,Female,10028,50.14,2
1,Workout_Type,Strength,5071,25.36,4
2,diet_type,Paleo,3403,17.02,6
3,cooking_method,Baked,2953,14.76,7
